In [1]:
import torch
torch.cuda.is_available()
print(torch.zeros(1).cuda())

tensor([0.], device='cuda:0')


In [1]:
import torch
import torch_geometric
import torch_geometric.nn as pyg_nn
from torch_sparse import SparseTensor, matmul
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import SAGEConv
print(torch_geometric.__version__)

import torch_scatter
import torch.nn as nn
import torch.nn.functional as F

import torch_geometric.utils as pyg_utils

from torch import Tensor
from typing import Union, Tuple, Optional
from torch_geometric.typing import (OptPairTensor, Adj, Size, NoneType, OptTensor)

from torch.nn import Parameter, Linear
from torch_sparse import SparseTensor, set_diag
from torch_geometric.nn.conv import MessagePassing
from torch_geometric.utils import remove_self_loops, add_self_loops, softmax

import networkx as nx
from deepsnap.hetero_graph import HeteroGraph
import copy
from copy import deepcopy
import pickle

import deepsnap
from deepsnap.graph import Graph
from deepsnap.batch import Batch
from deepsnap.dataset import GraphDataset
from deepsnap.hetero_gnn import forward_op
from deepsnap.hetero_graph import HeteroGraph
from sklearn.metrics import f1_score, roc_auc_score

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import json
from pathlib import Path as Data_Path
from os import listdir
from os.path import isfile, join
from itertools import combinations
from tqdm.notebook import tqdm

2.1.0


In [2]:
import graph_tool.all as gt
import json
print("graph-tool version: {}".format(gt.__version__.split(' ')[0]))

graph-tool version: 2.46


In [3]:
def json2graph(g, data_json):
    
    #add playlist nodes
    playlist_len=len(data_json['playlists'])
    playlist_attrs = []
    for i, playlist in enumerate(data_json['playlists']):
        playlist_attrs.append({'pl_ind': i, 'pl_name': playlist['name'], 'pl_type': 'playlist'})
 
    for node_attrs in playlist_attrs:
        v1 = g.add_vertex()
        for key, value in node_attrs.items():
            g.vp[key][v1] = value
            
    #add track nodes
    track_attrs = []
    # artist_attrs = []
    playlist_index=0
    edge_index=playlist_len
    # artist_edge_list=[]
    playlist_edge_list=[]
    track_index_dict={}
    # artist_track_dict={}
                               
    for playlist in data_json['playlists']:
        tracks_list = playlist['tracks']
        for track in tracks_list:
            dict_track={'tr_uri': track['track_uri'], 'tr_name': track['track_name'], 'tr_type': 'track'}
            if dict_track not in track_attrs:
                track_attrs.append(dict_track)
                track_index_dict[track['track_uri']]=edge_index
                edge_index+=1
            # dict_artist={'ar_uri': track['artist_uri'], 'ar_name': track['artist_name'], 'ar_type': 'artist'}
            # if dict_artist not in artist_attrs:
            #     artist_attrs.append(dict_artist)
            #     if track['artist_uri'] in artist_track_dict:
            #         artist_track_dict[track['artist_uri']].add(track['track_uri'])
            #     else:
            #         artist_track_dict[track['artist_uri']]={track['track_uri']}
            playlist_edge_list.append((playlist_index,edge_index))
            
        playlist_index+=1
 
    for node_attrs in track_attrs:
        v2 = g.add_vertex()
        for key, value in node_attrs.items():
            g.vp[key][v2] = value

    # artist_index=edge_index
    # for node_attrs in artist_attrs:
    #     v3 = g.add_vertex()
    #     for track in artist_track_dict[node_attrs['ar_uri']]:
    #         artist_edge_list.append((artist_index,track_index_dict[track]))
    #     for key, value in node_attrs.items():
    #         g.vp[key][v3] = value
    #     artist_index+=1
    # g.add_edge_list(artist_edge_list)
    g.add_edge_list(playlist_edge_list)


In [4]:
gt.openmp_enabled()
import multiprocessing

# Get the number of available CPUs
num_cpus = multiprocessing.cpu_count()
print(num_cpus)
gt.openmp_set_num_threads(num_cpus)

16


In [5]:
final_graph = gt.Graph(directed=False)

#playlist attributes
final_graph.vp['pl_ind'] = final_graph.new_vertex_property('string')
final_graph.vp['pl_name'] = final_graph.new_vertex_property('string')
final_graph.vp['pl_type'] = final_graph.new_vertex_property('string')
#track attributes
final_graph.vp['tr_uri'] = final_graph.new_vertex_property('string')
final_graph.vp['tr_name'] = final_graph.new_vertex_property('string')
final_graph.vp['tr_type'] = final_graph.new_vertex_property('string')
#artist attibutes
final_graph.vp['ar_uri'] = final_graph.new_vertex_property('string')
final_graph.vp['ar_name'] = final_graph.new_vertex_property('string')
final_graph.vp['ar_type'] = final_graph.new_vertex_property('string')


In [6]:
import os
folder_path = "../data_subset/"
file_paths = []
for root, dirs, files in os.walk(folder_path):
    for file in files:
        file_paths.append(os.path.join(root, file))

In [7]:
for file in file_paths:
    f = open(file)
    data = json.load(f)
    json2graph(final_graph, data)
    print(f'file {file} processed')

file ../data_subset/mpd.slice.0-999.json processed
file ../data_subset/mpd.slice.1000-1999.json processed


KeyboardInterrupt: 

In [ ]:
print("Number of vertices:", final_graph.num_vertices())
print("Number of edges:", final_graph.num_edges())

In [ ]:
count_pl = sum(1 for v in final_graph.vertices() if final_graph.vp.pl_type[v] == "playlist")
print(f'num_playlists: {count_pl}')
count_tr = sum(1 for v in final_graph.vertices() if final_graph.vp.tr_type[v] == "track")
print(f'num_tracks: {count_tr}')
count_ar = sum(1 for v in final_graph.vertices() if final_graph.vp.ar_type[v] == "artist")
print(f'num_artist: {count_ar}')

In [ ]:
attr_dict = {}
for node in G.vertices():
    attr = G.vp.name[node]
    if attr in attr_dict:
        attr_dict[attr].append(node)
    else:
        attr_dict[attr] = [node]

In [ ]:
# import gzip
# with gzip.open("graph_pta_100.pkl.gz", "wb") as f:
#     pickle.dump(final_graph, f)

In [29]:
with open("graph_pt_100.pickle", "wb") as f:
    pickle.dump(final_graph, f)

In [ ]:
#check property names

# vp1 = final_graph.vp["pl_name"]
# for v in final_graph.vertices():
#     prop_value = vp1[v]
#     print(prop_value)

In [ ]:
#separate pickling (if required)

# with open("final_graph.pickle", "rb") as f:
#     final_graph = pickle.load(f)
    # pl_name = pickle.load(f)
    # pl_type = pickle.load(f)
    # tr_uri = pickle.load(f)
    # tr_name = pickle.load(f)
    # tr_type = pickle.load(f)

# # set the vertex properties to the graph
# final_graph.vp["pl_name"] = pl_name
# final_graph.vp["pl_type"] = pl_type
# final_graph.vp["tr_uri"] = tr_uri
# final_graph.vp["tr_name"] = tr_name
# final_graph.vp["tr_type"] = tr_type